# Section 2 - Atmospheric data visualization

This section shows atmospheric variables downloaded from

## Necessary packages

In [1]:
import ipywidgets as widgets
from IPython.display import display

In [2]:
# import utilities functions
import os
cwd = os.getcwd()
cwd = '//'.join(cwd.split('//')[:-1])
import os
import sys
sys.path.insert(1, os.path.join(cwd, 'TropiDash'))

from utils_atm import *

In [3]:
# parameters to be imported from main widget
stdate = 0
# stdate = start_date_forecast.value # output from date selection widget

initial_lat_lon = (38, -90) #first cyclone position
final_lat_lon = (60, 90) #last cyclone position

In [4]:
# variable initialization
# variables to visualize
variables = ['msl', 'skt', 'tp','wind', '10fgg25'] #list of variables to visualize

# get mean coordinates to center the plot
mean_lat = (initial_lat_lon[0] - final_lat_lon[0])/2
mean_lon = (initial_lat_lon[1] - final_lat_lon[1])/2

coord = (mean_lat, mean_lon)

#steps to download
cyclone_days = 2
windprobsteps = [f"{12 * i}-{ 12 * i + 24}" for i in range(2*cyclone_days-1)]
windprobsteps.insert(0, "0-24")
stepsdict = {
               "base": list(np.arange(12, 240, 12)[0:2*cyclone_days]),
               "10fgg25": windprobsteps
            }

## To be updated when the starting date is changed

In [5]:
# Download
fnames = dwnl_atmdata_step(variables, stepsdict, stdate);
# Load
vardict = load_atmdata(variables, fnames)

## Plot and widget

In [6]:
# widget to select forecast step
vec_forecast = [f"{x}h from selected date" for x in stepsdict["base"]]
widget_sel_forecast = widgets.Dropdown(options = vec_forecast, description = 'Forecast')

In [7]:
# launch the interactive plot
widgets.interactive(plot_atmdata_step, 
                    step = widget_sel_forecast,
                    vardict = widgets.fixed(vardict),
                    coord =  widgets.fixed(coord), 
                    stepsdict =  widgets.fixed(stepsdict))

interactive(children=(Dropdown(description='Forecast', options=('12h from selected date', '24h from selected d…

In [ ]:
## find the issue

In [1]:
import branca.colormap as bc
import cfgrib
from datetime import datetime, timedelta
from ecmwf.opendata import Client
from ipyleaflet import Map, ColormapControl, LayersControl, basemaps
from ipyleaflet.velocity import Velocity
from IPython.display import display
from localtileserver import get_leaflet_tile_layer, TileClient
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import rasterio
import rioxarray as rxr
import xarray as xr

In [2]:
r = "C:/repos/TropiDash/tutorials/data/atm/10fgg25_20230907_time0_step0-24_enfo_ep.tiff"

In [4]:
var = "tp"
namedict = {
        "msl": "Mean sea level pressure [kPa]",
        "skt": "Skin temperature [°C]",
        "tp": "Total Precipitation [m]",
        "10fgg25": "Probability of 10 metre wind gust of at least 25 m/s [%]",
        "wind": "10 metre wind component [m/s]",
    }
m = Map(basemap = basemaps.Esri.WorldTopoMap, center = (0,0), zoom = 3)
client = TileClient(r)
t = get_leaflet_tile_layer(client, name = namedict[var], opacity = 0.7, projection = "EPSG:4326")
m.add_layer(t)
with rasterio.open(r) as raster:
    minv = "%.2f" % round(raster.read(1).ravel().min(), 1)
    maxv = "%.2f" % round(raster.read(1).ravel().max(), 1)
# add colorbar
cmap_control = ColormapControl(
                                caption = namedict[var],
                                # colormap = bc.StepColormap(palette),
                                value_min = float(minv),
                                value_max = float(maxv),
                                position = 'topright',
                                transparent_bg = True
                                )
m.add(cmap_control)
m

Map(center=[0, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text'…